In [1]:
dough_length = 500

biscuits = {
    0: {"length": 4, "value": 6, "defects": {"a": 4, "b": 2, "c": 3}, "id": 0},
    1: {"length": 8, "value": 12, "defects": {"a": 5, "b": 4, "c": 4}, "id": 1},
    2: {"length": 2, "value": 1, "defects": {"a": 1, "b": 2, "c": 1}, "id": 2},
    3: {"length": 5, "value": 8, "defects": {"a": 2, "b": 3, "c": 2}, "id": 3},
}

MAX_BISCUIT_LENGTH = max(b["length"] for b in biscuits.values())

In [2]:
with open("defects.csv") as f:
    lines = f.readlines()
    defects = [line.strip().split(",") for line in lines[1:]]

# Defects never occur at precise integer coordinates, so a defect will never be shared by two tiles.
defect_counts = {}
for x, y in defects:
    x = int(float(x))
    if x not in defect_counts:
        defect_counts[x] = {"a": 0, "b": 0, "c": 0}

    defect_counts[x][y] += 1

for x in range(dough_length):
    if x not in defect_counts:
        defect_counts[x] = {"a": 0, "b": 0, "c": 0}

print(len(defect_counts))

500


In [3]:
# Function to evaluate the total value of the biscuits in the current state
def evaluate_state(state):
    total_value = 0
    pos = 0
    while pos < dough_length:
        biscuit_idx = state[pos]
        if biscuit_idx != -1:
            biscuit = biscuits[biscuit_idx]
            total_value += biscuit["value"]
            pos += biscuit["length"]
        else:
            pos += 1
    return total_value

In [4]:
import math
import random


def apply_change(state, pos, new_biscuit):

    state[pos] = new_biscuit

    if new_biscuit == -1:
        return state

    if pos + biscuits[new_biscuit]["length"] - 1 >= dough_length:
        return None

    # Place new biscuit and remove overlapping biscuits
    for i in range(max(pos - MAX_BISCUIT_LENGTH, 0), pos):
        if state[i] != -1 and i + biscuits[state[i]]["length"] - 1 >= pos:
            state[i] = -1
    for i in range(pos + 1, min(pos + biscuits[new_biscuit]["length"], dough_length)):
        state[i] = -1

    biscuit_defects = {"a": 0, "b": 0, "c": 0}

    # Count biscuit defects
    for i in range(pos, pos + biscuits[new_biscuit]["length"]):
        for defect_type in ["a", "b", "c"]:
            biscuit_defects[defect_type] += defect_counts[i][defect_type]
    
    valid = True
    # Compare with threshold
    for defect_type in ["a", "b", "c"]:
        if biscuit_defects[defect_type] > biscuits[new_biscuit]["defects"][defect_type]:
            valid = False
            break
    
    return state if valid else None

# Simulated Annealing Algorithm
def simulated_annealing(state, temperature=1.0, cooling_rate=0.99, min_temperature=0.01):
    current_state = state
    current_value = evaluate_state(current_state)
    best_state = current_state.copy()
    best_value = current_value

    while temperature > min_temperature:
        new_state = current_state.copy()

        # Randomly modify the current solution
        pos = random.randint(0, dough_length - 1)
        choice_list = [-1, 0, 1, 2, 3]
        choice_list.remove(current_state[pos])
        new_biscuit = random.choice(choice_list)
        
        # Update state and calculate new value
        new_state = apply_change(new_state, pos, new_biscuit)
        if new_state is None:
            continue
        new_value = evaluate_state(new_state)

        # Calculate change in value
        value_change = new_value - current_value

        # Decide whether to accept the new solution
        if value_change >= 0 or random.uniform(0, 1) < math.exp(value_change / temperature):
            current_value = new_value
            if new_value > best_value:
                best_state = new_state.copy()
                best_value = new_value
            current_state = new_state.copy()

        # Decrease the temperature
        temperature *= cooling_rate

    return best_state, best_value

In [5]:
# Run the algorithm
starting_state = [-1] * dough_length
optimal_state, total_value = simulated_annealing(starting_state, temperature=1.0, cooling_rate=0.99999, min_temperature=0.001)
print("Optimal State:", optimal_state)
print("Total Value:", total_value)


Optimal State: [1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 2, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, 0, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 2, -1, 0, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, 2, -1, 1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1

In [6]:
from utils import check_solution
# Build and check solution
solution = [(x, optimal_state[x]) for x in range(dough_length) if optimal_state[x] != -1]
print(check_solution(solution))

(727, [(0, 1), (8, 1), (16, 0), (20, 1), (28, 1), (36, 0), (40, 1), (48, 2), (51, 1), (59, 1), (67, 3), (73, 1), (81, 1), (89, 1), (100, 3), (105, 1), (114, 0), (118, 0), (122, 1), (130, 1), (138, 2), (140, 0), (144, 1), (152, 3), (158, 1), (168, 1), (176, 1), (185, 2), (187, 1), (195, 3), (200, 1), (208, 1), (216, 1), (224, 1), (232, 3), (237, 1), (245, 1), (253, 1), (261, 1), (270, 1), (278, 1), (286, 1), (294, 1), (302, 1), (310, 1), (318, 1), (326, 1), (334, 1), (342, 0), (346, 1), (354, 1), (362, 0), (366, 0), (370, 0), (374, 1), (382, 1), (390, 1), (398, 1), (406, 2), (408, 3), (413, 1), (421, 1), (429, 1), (437, 1), (445, 1), (453, 2), (455, 1), (463, 1), (471, 1), (479, 1), (487, 3), (492, 1)], True)


In [17]:
from utils import save_solution
# Save solution
save_solution(solution, "solution_simulated_annealing.txt")